In [1]:
from pandas.tseries.offsets import DateOffset
import ee
import yaml
import time
import os
import json
import pandas as pd
from utils import exportImageToGDrive,exportImageToGCS
from download_sen12 import *

ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][0]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)

project-canopy-temp-2


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:444: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [3]:
sensor['bands']

['TCI_R', 'TCI_G', 'TCI_B']

In [4]:
source

{'name': ['dynamic_date_range_v4_secondary_sort_using_area_full_band_output'],
 'start_date': '2019-01-01',
 'end_date': '2020-12-31',
 'geometry': 'point',
 'size': 20000,
 'resolution': 10,
 'sort_by': 'name',
 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
 'sensors': [0, 1, 2]}

In [4]:
start = pd.to_datetime('01-01-2020')

start

Timestamp('2020-01-01 00:00:00')

In [3]:
def import_aois(csv_loc, Full_Congo_Pull=False, start_date=None,
                end_date=None, days_duration=90, poly_start=0, poly_limit=None):
    features = []
    polygons = []
    day_offset = days_duration / 2
    start_end_list = []
    
    if Full_Congo_Pull:
        with open(csv_loc,"r",encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            for geometry in data["features"]:
                polygon = geometry["geometry"]["coordinates"][0][0]
                poly_obj = ee.Geometry.Polygon(polygon)
                feature = ee.Feature(poly_obj)
                features.append(feature)
                
            original_date = pd.to_datetime('01-01-2020')
            start = (original_date + DateOffset(days=-30))
            end = (original_date + DateOffset(days=30))
            start_date = str(start)[:10]
            end_date = str(end)[:10]
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': 30,
                'area': 1
            }
            
            start_end_list = [date_dict] * len(features)
            
            return ee.FeatureCollection(features), start_end_list
            
    else:
        feature_id = poly_start
        
        if poly_limit:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1), nrows=poly_limit)
        else:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1))

        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined","tile date","area (km2)"]]
        df_labels["tile date"] = pd.to_datetime(df_labels["tile date"])
        start = (df_labels["tile date"] + DateOffset(days=-day_offset))
        end = (df_labels["tile date"] + DateOffset(days=day_offset))
        for i in range(len(start)):
            start_date = str(start[i])[:10]
            end_date = str(end[i])[:10]
            original_date = df_labels.loc[i, 'tile date']
            area = df_labels.loc[i, 'area (km2)']
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': day_offset,
                'area': area
            }
            start_end_list.append(date_dict)

        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])
            
        for poly in polygons:
            # create an roi. first item in Misha's label list
            feature_id += 1 
            # create geometry object, create feature object, append to features list for feature collection creation 
            polys = ee.Geometry.Polygon(poly)
            feature = ee.Feature(polys,{"name":feature_id})
            features.append(feature)
            
        return ee.FeatureCollection(features),start_end_list,df_labels


# ### CHANGE BELOW PATH ###
zhenya_label_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
david__label_path = 'D:/canopy_data/csvs/polygons_101320.csv'
zhenya_tilelist_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tiles_polygons_full.csv"
david_grid_path = 'D:/canopy_data/Geometry/Congo_Basin_Shapefile_Grid_v1.geojson'

In [4]:
fc,date_range_list = import_aois(david_grid_path, Full_Congo_Pull=True)

In [ ]:
date_range_list[0:5]

In [12]:
fc

In [13]:
fc.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[7.774437227891959, 5.590436336],
    [7.789316305, 5.590436336],
    [7.789316305, 5.607344378168228],
    [7.774437227891959, 5.590436336]]]},
 'id': '0',
 'properties': {}}

In [8]:
exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047
processing polygon 2 of 5047
processing polygon 3 of 5047
processing polygon 4 of 5047
processing polygon 5 of 5047
processing polygon 6 of 5047
processing polygon 7 of 5047
processing polygon 8 of 5047
processing polygon 9 of 5047
processing polygon 10 of 5047
processing polygon 11 of 5047
processing polygon 12 of 5047
processing polygon 13 of 5047
processing polygon 14 of 5047
processing polygon 15 of 5047
processing polygon 16 of 5047
processing polygon 17 of 5047
processing polygon 18 of 5047
processing polygon 19 of 5047
processing polygon 20 of 5047
processing polygon 21 of 5047
processing polygon 22 of 5047
processing polygon 23 of 5047
processing polygon 24 of 5047
processing polygon 25 of 5047
processing polygon 26 of 5047
processing polygon 27 of 5047
processing polygon 28 of 5047
processing polygon 29 of 5047
processing polygon 30 of 5047
processing polygon 31 of 5047
processing polygon 32 of 5047
processing polygon

EEException: Computation timed out.

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [8]:
## trying tile scale 32 ##

exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047


EEException: Collection.reduceColumns: Error in map(ID=20191206T094259_20191206T095558_T32NLM):
ReduceRegion.AggregationContainer: Valid tileScales are 1 to 16.

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [9]:
import time


for i in range(10):
    if i == 5:
        print('pause')
        time.sleep(5)
        
    print(i)

0
1
2
3
4
pause
5
6
7
8
9


In [10]:
for i in (-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5):
    try:
        a = 10 / i
        print(a)
    except Exception as e:
        print(e)
        time.sleep(5)

-2.0
-2.5
-3.3333333333333335
-5.0
-10.0
division by zero
10.0
5.0
3.3333333333333335
2.5
2.0


In [12]:
def division(top, bot):
    return top / bot

for i in (-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5):
    try:
        a = division(10, i)
        print(a)
    except Exception as e:
        print(e)
        time.sleep(5)
        a = division(10, i+0.5)
        print(a)

-2.0
-2.5
-3.3333333333333335
-5.0
-10.0
division by zero
20.0
10.0
5.0
3.3333333333333335
2.5
2.0


In [13]:
params = {'top': 10, 'bot': 5}

division(**params)

2.0

In [7]:
## testing try/except loop

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047
processing polygon 2 of 5047
processing polygon 3 of 5047
processing polygon 4 of 5047
processing polygon 5 of 5047
processing polygon 6 of 5047
processing polygon 7 of 5047
processing polygon 8 of 5047
processing polygon 9 of 5047
processing polygon 10 of 5047
processing polygon 11 of 5047
processing polygon 12 of 5047
processing polygon 13 of 5047
processing polygon 14 of 5047
processing polygon 15 of 5047
processing polygon 16 of 5047
processing polygon 17 of 5047
processing polygon 18 of 5047
processing polygon 19 of 5047
processing polygon 20 of 5047
processing polygon 21 of 5047
processing polygon 22 of 5047
processing polygon 23 of 5047
processing polygon 24 of 5047
processing polygon 25 of 5047
processing polygon 26 of 5047
processing polygon 27 of 5047
processing polygon 28 of 5047
processing polygon 29 of 5047
processing polygon 30 of 5047
processing polygon 31 of 5047
processing polygon 32 of 5047
processing polygon

Traceback (most recent call last):
  File "C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\_ffi\loop.py", line 269, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
KeyboardInterrupt
2020-12-08T22:30:53Z


KeyboardInterrupt: 

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [7]:
## testing a one-hour wait

offset_dict = {
    30: 'two years'
}

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047
processing polygon 2 of 5047
processing polygon 3 of 5047
processing polygon 4 of 5047
processing polygon 5 of 5047
processing polygon 6 of 5047
processing polygon 7 of 5047
processing polygon 8 of 5047
processing polygon 9 of 5047
processing polygon 10 of 5047
processing polygon 11 of 5047
processing polygon 12 of 5047
processing polygon 13 of 5047
processing polygon 14 of 5047
processing polygon 15 of 5047
processing polygon 16 of 5047
processing polygon 17 of 5047
processing polygon 18 of 5047
processing polygon 19 of 5047
processing polygon 20 of 5047
processing polygon 21 of 5047
processing polygon 22 of 5047
processing polygon 23 of 5047
processing polygon 24 of 5047
processing polygon 25 of 5047
processing polygon 26 of 5047
processing polygon 27 of 5047
processing polygon 28 of 5047
processing polygon 29 of 5047
processing polygon 30 of 5047
processing polygon 31 of 5047
processing polygon 32 of 5047
processing polygon

Traceback (most recent call last):
  File "C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\_ffi\loop.py", line 269, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
KeyboardInterrupt
2020-12-09T15:33:10Z


KeyboardInterrupt: 

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
## Misha polygon pull

config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][1]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)
print(source)
print(sensor)

project-canopy-temp-2
{'name': ['dynamic_date_range_v4_secondary_sort_using_area_full_band_output'], 'start_date': '2019-01-01', 'end_date': '2020-12-31', 'geometry': 'point', 'size': 20000, 'resolution': 10, 'sort_by': 'name', 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI', 'sensors': [0, 1, 2]}
{'name': 'COPERNICUS/S2_SR', 'prefix': 'S2_CloudFree', 'type': 'opt', 'bands': ['B2', 'B3', 'B4', 'B8', 'B8A', 'TCI_R', 'TCI_G', 'TCI_B', 'NDVI'], 'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]}


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:442: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [4]:
fc,date_range_list,df_labels = import_aois(david__label_path)

In [7]:
offset_dict = {
    45: 90,
    90: 180,
    180: 'two years'
}

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list, loop_start=81
)

101 features have been loaded
processing polygon 82 of 101
processing polygon 83 of 101
processing polygon 84 of 101
processing polygon 85 of 101
processing polygon 86 of 101
processing polygon 87 of 101
processing polygon 88 of 101
processing polygon 89 of 101
processing polygon 90 of 101
processing polygon 91 of 101
processing polygon 92 of 101
processing polygon 93 of 101
processing polygon 94 of 101
processing polygon 95 of 101
processing polygon 96 of 101
processing polygon 97 of 101
processing polygon 98 of 101
processing polygon 99 of 101
processing polygon 100 of 101
processing polygon 101 of 101


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
## testing a one-hour wait, with a 30-second delay between exports

offset_dict = {
    30: 'two years'
}

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047
Polygon 1 processed; please wait
processing polygon 2 of 5047
Polygon 2 processed; please wait
processing polygon 3 of 5047
Polygon 3 processed; please wait
processing polygon 4 of 5047
Polygon 4 processed; please wait
processing polygon 5 of 5047
Polygon 5 processed; please wait
processing polygon 6 of 5047
Polygon 6 processed; please wait
processing polygon 7 of 5047
Polygon 7 processed; please wait
processing polygon 8 of 5047
Polygon 8 processed; please wait
processing polygon 9 of 5047
Polygon 9 processed; please wait
processing polygon 10 of 5047
Polygon 10 processed; please wait
processing polygon 11 of 5047
Polygon 11 processed; please wait
processing polygon 12 of 5047
Polygon 12 processed; please wait
processing polygon 13 of 5047
Polygon 13 processed; please wait
processing polygon 14 of 5047
Polygon 14 processed; please wait
processing polygon 15 of 5047
Polygon 15 processed; please wait
processing polygon 16 of 50